In [1]:
from CheckGraphic import CheckGraphicID

import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pymongo
import bson
import matplotlib.pyplot as plt
import matplotlib
import random
from itertools import permutations
from IPython.display import Image 
from IPython.core.display import Image, display
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [4]:
db = db_client["tweets_pipeline_v2"].find(
                                           {"status":"pipelined",
                                           "source":"def"},
    no_cursor_timeout=True)

In [5]:
for i in db:
    print(i)
    break

{'_id': ObjectId('5f113253684654236514c9e8'), 'search_text': 'range resources "market share" 2020', 'exec_date': datetime.datetime(2020, 7, 17, 4, 57, 48, 13000), 'img_urls': 'https://www.sec.gov/Archives/edgar/data/315852/000130817918000145/lrrc047.jpg', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 7, 17, 4, 57, 48, 13000), 'source': 'def', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'pipelined', 'extracted_text': 'TOTAL PROVED RESERVES COSTS PER MCFE (BCFE) $2.19 15,262 $1.78 $1.68 12,072 9,892 2015 2016 2017 2015 2016 2017 DRILL BIT FINDING COSTS* PRODUCTION GROWTH PER SHARE (PER MCFE) (DEBT ADJUSTED) 14% $0.85 $0.36 4% $0.33 0% 2015 2016 2017 2015 2016 2017 * Excluding acquisitions RESERVE GROWTH PER SHARE (DEBT ADJUSTED) EBITDAX (IN MILLIONS) 24% $1,103 $897 $723 2015 2016 2017 2015 2016 2017 -6% -6%', 'bert_tags': [], 'industries_from_bert': ['Energy sources'], 'shifts_from_bert': [], 'trasher': [], 'pipelined_ts': datetime.datetime(2020, 7, 17, 5, 8, 35, 296000), 

In [11]:
# for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"pipelined"},
# #                                                            {"status":"graphicone_feed"}
#                                                       ],
#                                                        'source': 'sec',
# #                                                    'approved_timestamp': {'$gte': end_date, '$lte': start_date},
#                                                    }):
#     print(i['_id'])
#     print(i['entity_prepared']+i["bert_tags"])    
#     CheckGraphicID(i['_id'])



In [12]:
import urllib.request
from PIL import Image
# url = 'http://www.example.com/my_image_is_not_your_image.png'
# image = Image.open(urllib.request.urlopen(url))

In [13]:
def sizeFilter(size):
    width, height = image.size
    if width < 350 or height < 350:
        return False
    else:
        return True

In [14]:
def small_size(_id):
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                               {"$set":{
                                                   'status': "small_size",
                                                       }})      

In [15]:
db = db_client["tweets_pipeline_v2"].find(
                                           {"status":"pipelined",
                                           "source":"def"},
    no_cursor_timeout=True)

db.count()

2104

In [16]:
from tqdm import tqdm

In [17]:
for i in tqdm(db, total = db.count()):
    try:
        image = Image.open(urllib.request.urlopen(i['img_urls']))
        if not sizeFilter(image.size):
            try:
                small_size(str(i['_id']))
    #             CheckGraphicID(i['_id'])
            except Exception as err:
                pass
    except Exception as err:
        pass

             
    
#     try:
#         image = Image.open(urllib.request.urlopen(i['img_urls']))
#         if not sizeFilter(image.size):
#             print(image.size)
#             print(i['_id'])
#             print(i["bert_tags"])
#     #         print(i['entity_prepared']+i["bert_tags"])    
#             CheckGraphicID(i['_id'])
#     except Exception as err:
#         pass
# #     break

100%|██████████| 2104/2104 [09:32<00:00,  3.68it/s]


In [18]:
db_client["tweets_pipeline_v2"].find({'status': "small_size"}, no_cursor_timeout=True).count()

2281

In [ ]:
a = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(str("5f1385be3e25deec88b6287e"))})
Image.open(urllib.request.urlopen(a['img_urls'])).size

In [ ]:
# for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"trasher"},
# #                                                            {"status":"graphicone_feed"}
#                                                       ],
#                                                        'source': 'sec',
# #                                                    'approved_timestamp': {'$gte': end_date, '$lte': start_date},
#                                                    }):
    

#     i['img_urls']

In [114]:
db = db_client["tweets_pipeline_v2"].find(
                                           {"status":"pipelined",
                                           "source":"def"},
    no_cursor_timeout=True)

db.count()


tagsCollector = []
for i in db:
    tags = i["entity_prepared"]+i["bert_tags"]
    tags = list(set(tags))
    tagsCollector += tags

In [115]:
pd.DataFrame.from_dict(dict(Counter(tagsCollector).most_common()), orient='index').to_excel("bd_search_graphs.xlsx")

In [112]:
db = db_client["tweets_pipeline_v2"].find(
                                        {
        "$or": [{"status":"graphicone_search"}, {"status":"graphicone_feed"}],
                                        },
    no_cursor_timeout=True)

db.count()


tagsCollector = []
for i in db:
    tagsCollector += i["grafeed_confirmed"]

In [113]:
pd.DataFrame.from_dict(dict(Counter(tagsCollector).most_common()), orient='index').to_excel("all_graphs.xlsx")

In [10]:
a = set(["employment", "labor"])
a

{'employment', 'labor'}

In [14]:
b = set(["employment", "aasdsad", "asdasdsa"])

In [15]:
a.issubset(b)

False

In [9]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")

tags_by_tag_old = client.get(data_as_key)[-1]['tags']

TimeoutError: (9, 'Client timeout: socket=30000 total=1000 iterations=1 lastNode=35.228.136.58:3000', 'src/main/client/get.c', 113, False)

In [ ]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())
    return list(set(result))

In [ ]:
collector = []
collector_sub = []
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},
                                                       {"status":"graphicone_feed"}],
                                               }):
    if "grafeed_confirmed" in i:
        analyst = set(i["grafeed_confirmed"])
        collector += dictImplementation(analyst)
#         if set(["employment", "labor"]).issubset(analyst):
#             collector_sub += analyst




In [ ]:
from collections import Counter

In [10]:
pd.DataFrame.from_dict(dict(Counter(collector)), orient="index").to_excel("anzelika_total.xlsx")

In [3]:
# pd.DataFrame.from_dict(dict(Counter(collector_sub)), orient="index").to_excel("anzelika_subset.xlsx")

In [ ]:
cities = pd.read_csv("worldcities.csv")
cities = cities[(cities["population"]/1000000)>=1]["city_ascii"]
cities_set = set([city.lower() for city in cities.values])

tags_dictionary_set_without_cities = tags_dictionary_set_without_countries - cities_set
len(tags_dictionary_set_without_cities)

# cities
# countries_set = 
# print(len(countries_set))

In [ ]:
countries = pd.read_csv("countries-data.csv")["Name"]
countries_set = set([country.lower() for country in countries.values])
print(len(countries_set))

tags_dictionary_set_without_countries = tags_dictionary_set_without_securities - countries_set
len(tags_dictionary_set_without_countries)